In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error


In [ ]:
df = pd.read_csv('split/training_split.csv')
df_val = pd.read_csv('split/validation_split.csv')

In [ ]:
X = df.drop("ActualTOW", axis=1)  # Features
y = df["ActualTOW"]  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = X_train.join(y_train)

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
train_data.hist(figsize=(15,12))

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(train_data, annot=True,cmap= 'coolwarm')

In [ ]:
# TODO: preprocess the data for better results 

# train_data['ActualFlightTime'] = np.log(train_data['ActualFlightTime']+1)
# train_data['ActualTotalFuel'] = np.log(train_data['ActualTotalFuel']+1)
# train_data['FLownPassengers'] = np.log(train_data['FLownPassengers']+1)
# train_data['BagsCount'] = np.log(train_data['BagsCount']+1)
# train_data['FlightBagsWeight'] = np.log(train_data['FlightBagsWeight']+1)

# test_data = X_test.join(Y_test)
# test_data['ActualFlightTime'] = np.log(test_data['ActualFlightTime']+1)
# test_data['ActualTotalFuel'] = np.log(test_data['ActualTotalFuel']+1)
# test_data['FLownPassengers'] = np.log(test_data['FLownPassengers']+1)
# test_data['BagsCount'] = np.log(test_data['BagsCount']+1)
# test_data['FlightBagsWeight'] = np.log(test_data['FlightBagsWeight']+1)

In [ ]:
# train_data.hist(figsize=(15,12))

In [ ]:
# plt.figure(figsize=(15,8))
# sns.scatterplot(x='BagsCount', y='ActualTotalFuel', data=train_data, hue= 'ActualTOW', palette='coolwarm')

In [ ]:
X_train, Y_train = train_data.drop(['ActualTOW'], axis=1), train_data['ActualTOW']

reg = LinearRegression().fit(X_train, Y_train)
reg.score(X_train, Y_train)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)
y_pred = model.predict(X_train)
model.score(X_train, Y_train)
# mse = mean_squared_error(Y_train, y_pred)
# print("MSE: %.2f" % mse)

In [ ]:
# Create the output directory if it doesn't exist
output_directory = 'output'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print("Output directory created successfully.")

# save data to csv file
df_val['PredictedActualTOW'] = y_pred
df_val.to_csv('validation_with_predictions.csv', index=False)